Mount Google Drive & Set Paths

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Define your dataset path
data_dir = "/content/drive/MyDrive/Medical/chest_xray"


Mounted at /content/drive


Display a Sample Image

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

sample_path = os.path.join(data_dir, "train/PNEUMONIA/person1_bacteria_1.jpeg")
image = cv2.imread(sample_path, cv2.IMREAD_GRAYSCALE)

plt.imshow(image, cmap='gray')
plt.title("Sample X-Ray Image")
plt.axis('off')
plt.show()


Preprocess with ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    os.path.join(data_dir, "train"),
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary'
)

val_data = datagen.flow_from_directory(
    os.path.join(data_dir, "val"),
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary'
)

test_data = datagen.flow_from_directory(
    os.path.join(data_dir, "test"),
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Build and Train CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)


Evaluate Model Accuracy

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

y_true = test_data.classes
y_pred = model.predict(test_data)
y_pred = np.round(y_pred).flatten()

acc = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print(f"Model Accuracy: {acc * 100:.2f}%")
print("Confusion Matrix:\n", cm)


Save the Trained Model

In [ ]:
model.save("medical_ai_model.h5")

# Optional: Save it to Google Drive
!cp medical_ai_model.h5 /content/drive/MyDrive/Medical/
